In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
base_path = '/content/drive/MyDrive/bird-sounds-rnn/raw'
os.makedirs(base_path, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install xeno-canto

  Preparing metadata (setup.py) ... done
  Created wheel for xeno-canto: filename=xeno_canto-3.0-py3-none-any.whl size=8705 sha256=418437ef266f1caa4d53cd49166a9b8cd9107dfc72e79e1e54a8581c8e6154b1
  Stored in directory: /root/.cache/pip/wheels/ed/5d/b1/d820dc701c129c102c41e9d65288c08772ecd6aee382ae1d57
Successfully built xeno-canto


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import requests
import time
import json  # If loading from string; or open file

base_path = '/content/drive/MyDrive/bird-sounds-rnn/raw/'  # Adjust if needed
os.makedirs(base_path, exist_ok=True)

# Option 1: If you saved the JSON as data_file.txt in Drive
with open('/content/data_file.txt', 'r') as f:
    data = json.load(f)
xc_ids = list(data.keys())  # One line to get all 833 IDs!

# Option 2: If pasting the JSON directly (for quick test)
# data = json.loads("""{your full JSON here}""")
# xc_ids = list(data.keys())

print(f"Loaded {len(xc_ids)} IDs – ready to download!")

for i, xc_id in enumerate(xc_ids):
    filename = f"XC{xc_id}.mp3"
    filepath = os.path.join(base_path, filename)
    if os.path.exists(filepath):
        print(f"[{i+1}/{len(xc_ids)}] Skipping existing {filename}")
        continue
    url = f"https://xeno-canto.org/{xc_id}/download"
    try:
        resp = requests.get(url, timeout=90)
        if resp.status_code == 200:
            with open(filepath, 'wb') as f:
                f.write(resp.content)
            size_mb = os.path.getsize(filepath) / 1e6
            print(f"[{i+1}/{len(xc_ids)}] Downloaded {filename} ({size_mb:.1f} MB)")
        else:
            print(f"[{i+1}/{len(xc_ids)}] Failed {xc_id}: HTTP {resp.status_code}")
    except Exception as e:
        print(f"[{i+1}/{len(xc_ids)}] Error {xc_id}: {e}")
      # Polite delay – increase if rate-limited

print("Download complete! All files in your raw/ folder.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 833 IDs – ready to download!
[1/833] Skipping existing XC1065199.mp3
[2/833] Skipping existing XC934128.mp3
[3/833] Skipping existing XC928661.mp3
[4/833] Skipping existing XC928658.mp3
[5/833] Skipping existing XC667381.mp3
[6/833] Skipping existing XC577361.mp3
[7/833] Skipping existing XC577358.mp3
[8/833] Skipping existing XC577357.mp3
[9/833] Skipping existing XC571156.mp3
[10/833] Skipping existing XC195759.mp3
[11/833] Skipping existing XC312771.mp3
[12/833] Skipping existing XC574340.mp3
[13/833] Skipping existing XC574339.mp3
[14/833] Downloaded XC574336.mp3 (0.3 MB)
[15/833] Downloaded XC574335.mp3 (0.7 MB)
[16/833] Downloaded XC574331.mp3 (0.2 MB)
[17/833] Downloaded XC574328.mp3 (0.1 MB)
[18/833] Downloaded XC574318.mp3 (0.6 MB)
[19/833] Downloaded XC315635.mp3 (0.1 MB)
[20/833] Downloaded XC332453.mp3 (0.1 MB)
[21/833] Downloaded XC662750.

In [ ]:
import pandas as pd
import json

df = pd.read_csv('/content/xenocanto.csv')
id = df['Catalogue number']
name = df['Scientific name']

list_name = {}

for i in range(len(id)):
  list_name.update({int(id[i]): name[i]})
print(list_name)

with open('/content/data_file.txt', 'w') as file:
    json.dump(list_name, file, indent=4)

{1065199: 'Funambulus palmarum', 934128: 'Tarbinskiellus portentosus', 928661: 'Duttaphrynus melanostictus', 928658: 'Duttaphrynus melanostictus', 667381: 'Dendrocygna javanica', 577361: 'Tadorna ferruginea', 577358: 'Tadorna ferruginea', 577357: 'Tadorna ferruginea', 571156: 'Anas poecilorhyncha', 195759: 'Anas poecilorhyncha', 312771: 'Galloperdix spadicea', 574340: 'Gallus sonneratii', 574339: 'Gallus sonneratii', 574336: 'Gallus sonneratii', 574335: 'Gallus sonneratii', 574331: 'Gallus sonneratii', 574328: 'Gallus sonneratii', 574318: 'Gallus sonneratii', 315635: 'Gallus sonneratii', 332453: 'Ortygornis pondicerianus', 662750: 'Francolinus pictus', 338333: 'Francolinus pictus', 747786: 'Francolinus pictus', 592695: 'Francolinus pictus', 571151: 'Francolinus pictus', 428896: 'Francolinus pictus', 672092: 'Coturnix coromandelica', 592654: 'Coturnix coromandelica', 592653: 'Coturnix coromandelica', 592651: 'Coturnix coromandelica', 592633: 'Coturnix coromandelica', 592632: 'Coturnix c

In [1]:
!pip install torchcodec

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
import torch
import torchaudio
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# ================== SETTINGS ==================
raw_dir = '/content/drive/MyDrive/bird-sounds-rnn/raw/'          # Folder with your 833 XC*.mp3 files
spec_dir = '/content/drive/MyDrive/bird-sounds-rnn/mel_specs/all/'  # All spectrograms go here
os.makedirs(spec_dir, exist_ok=True)

# Optional: Load your JSON mapping if you want species info later
# import json
# with open('/content/drive/MyDrive/BirdSoundMumbai/data_file.txt', 'r') as f:
#     data = json.load(f)

# ================== GPU SETUP ==================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")  # Should show cuda with T4 selected

# ================== MEL TRANSFORM (GPU) ==================
mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=22050,
    n_fft=512,
    win_length=512,
    hop_length=256,          # 50% overlap → ~431 frames for 5s
    f_min=50,
    f_max=11000,
    n_mels=128,
    norm='slaney',
    mel_scale='slaney'
).to(device)

db_transform = torchaudio.transforms.AmplitudeToDB(stype='power', top_db=80).to(device)

full_transform = torch.nn.Sequential(mel_transform, db_transform)

# ================== PARAMETERS ==================
target_sr = 22050
target_len = target_sr * 5   # Fixed 5-second clips

total_specs = 0
print("Starting conversion of ALL 833 recordings to Mel spectrograms (5s clips with overlap)...")

for filename in tqdm(sorted(os.listdir(raw_dir))):
    if not filename.lower().endswith('.mp3'):
        continue

    filepath = os.path.join(raw_dir, filename)

    try:
        # Load audio
        waveform, sr = torchaudio.load(filepath)

        # Resample to 22050 Hz
        if sr != target_sr:
            waveform = torchaudio.functional.resample(waveform, orig_freq=sr, new_freq=target_sr)

        # Convert to mono
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        total_samples = waveform.shape[1]
        start_sample = 0
        clip_idx = 0

        while start_sample < total_samples:
            end_sample = min(start_sample + target_len, total_samples)
            segment = waveform[:, start_sample:end_sample]

            # Zero-pad if shorter than 5s
            if segment.shape[1] < target_len:
                pad = target_len - segment.shape[1]
                segment = torch.nn.functional.pad(segment, (0, pad))

            # Move to GPU, compute Mel spec
            segment = segment.to(device)
            with torch.no_grad():
                mel = full_transform(segment)          # (1, 128, time)

            mel = mel.squeeze(0).cpu()                     # (128, time) on CPU

            # Normalize to 0-255 for image saving
            mel_min = mel.min()
            mel_max = mel.max()
            if mel_max > mel_min:
                mel = (mel - mel_min) / (mel_max - mel_min)
            mel_img = (mel.numpy() * 255).astype('uint8')

            # Save as PNG
            base_name = os.path.splitext(filename)[0]
            save_name = f"{base_name}_part{clip_idx}.png"
            save_path = os.path.join(spec_dir, save_name)

            plt.imsave(save_path, mel_img, cmap='magma', origin='lower')

            total_specs += 1
            clip_idx += 1

            # 50% overlap for more training examples (skip if near end)
            if total_samples - end_sample > target_len // 2:
                start_sample += target_len // 2
            else:
                break

    except Exception as e:
        print(f"\nError processing {filename}: {e}")

print(f"\nConversion complete!")
print(f"Generated {total_specs} Mel spectrogram images (5s clips).")
print(f"All saved in: {spec_dir}")
print("Each image is ~128 × 431 pixels – perfect for CNNs or patching into Transformers.")
print("Next step: Build a labeled Dataset using your species map and start training!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
Starting conversion of ALL 833 recordings to Mel spectrograms (5s clips with overlap)...


  1%|          | 7/833 [01:40<4:24:13, 19.19s/it]


Error processing XC1065186.mp3: Failed to create AudioDecoder for /content/drive/MyDrive/bird-sounds-rnn/raw/XC1065186.mp3: Could not open input file: /content/drive/MyDrive/bird-sounds-rnn/raw/XC1065186.mp3 Invalid argument


  3%|▎         | 27/833 [02:16<06:29,  2.07it/s]


Error processing XC124743.mp3: Failed to create AudioDecoder for /content/drive/MyDrive/bird-sounds-rnn/raw/XC124743.mp3: Could not open input file: /content/drive/MyDrive/bird-sounds-rnn/raw/XC124743.mp3 Invalid argument


...


Error processing XC785416.mp3: Failed to create AudioDecoder for /content/drive/MyDrive/bird-sounds-rnn/raw/XC785416.mp3: Could not open input file: /content/drive/MyDrive/bird-sounds-rnn/raw/XC785416.mp3 Invalid argument


 99%|█████████▊| 821/833 [08:31<00:06,  1.81it/s]


Error processing XC892006.mp3: Failed to create AudioDecoder for /content/drive/MyDrive/bird-sounds-rnn/raw/XC892006.mp3: Could not open input file: /content/drive/MyDrive/bird-sounds-rnn/raw/XC892006.mp3 Invalid argument


100%|██████████| 833/833 [08:53<00:00,  1.56it/s]


Conversion complete!
Generated 8037 Mel spectrogram images (5s clips).
All saved in: /content/drive/MyDrive/bird-sounds-rnn/mel_specs/all/
Each image is ~128 × 431 pixels – perfect for CNNs or patching into Transformers.
Next step: Build a labeled Dataset using your species map and start training!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm
import numpy as np

# ================== PATHS ==================
spec_dir = '/content/drive/MyDrive/bird-sounds-rnn/mel_specs/all/'
json_path = '/content/data_file.txt'  # Adjust if path different

with open(json_path, 'r') as f:
    xc_to_species = json.load(f)

# ================== DYNAMIC LABEL MAP FOR ALL BIRDS ==================
unique_species = sorted(set(xc_to_species.values()))

label_map = {species: idx for idx, species in enumerate(unique_species)}
num_classes = len(unique_species)
class_names = unique_species  # List of all scientific names

print(f"Found {num_classes} unique bird species!")
print("Classes:", class_names)  # Will print all ~200+ species

# ================== CUSTOM DATASET ==================
class BirdSpecDataset(Dataset):
    def __init__(self, spec_dir, xc_to_species, label_map, transform=None):
        self.spec_dir = spec_dir
        self.xc_to_species = xc_to_species
        self.label_map = label_map
        self.transform = transform

        self.files = [f for f in os.listdir(spec_dir) if f.endswith('.png')]
        self.labels = []
        for f in self.files:
            xc_id = f.split('_')[0].replace('XC', '')
            sci_name = xc_to_species.get(xc_id, "unknown")  # fallback
            label = label_map.get(sci_name, label_map.get("unknown", 0))  # fallback to 0
            self.labels.append(label)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.spec_dir, self.files[idx])
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# ================== TRANSFORMS ==================
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ================== FULL DATASET & SPLIT ==================
full_dataset = BirdSpecDataset(spec_dir, xc_to_species, label_map)

train_size = int(0.85 * len(full_dataset))
val_size = len(full_dataset) - train_size
generator = torch.Generator().manual_seed(42)

split = torch.utils.data.random_split(
    range(len(full_dataset)),
    [train_size, val_size],
    generator=generator
)

train_indices = split[0].indices
val_indices = split[1].indices

train_dataset = BirdSpecDataset(spec_dir, xc_to_species, label_map, transform=train_transform)
train_dataset.files = [full_dataset.files[i] for i in train_indices]
train_dataset.labels = [full_dataset.labels[i] for i in train_indices]

val_dataset = BirdSpecDataset(spec_dir, xc_to_species, label_map, transform=val_transform)
val_dataset.files = [full_dataset.files[i] for i in val_indices]
val_dataset.labels = [full_dataset.labels[i] for i in val_indices]

# ================== BALANCED SAMPLER (for multi-class) ==================
train_labels_np = np.array(train_dataset.labels)
class_counts = np.bincount(train_labels_np, minlength=num_classes)
class_weights = 1. / (class_counts + 1e-6)
sample_weights = class_weights[train_labels_np]

sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

print(f"Total images: {len(full_dataset)} | Train: {len(train_dataset)} | Val: {len(val_dataset)}")
print(f"Training on ALL {num_classes} bird species dynamically!")

# ================== MODEL ==================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

# ================== TRAINING LOOP ==================
num_epochs = 30  # More epochs for many classes
best_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total if total > 0 else 0
    print(f"\nEpoch {epoch+1} - Loss: {running_loss/len(train_loader):.4f} - Val Acc: {acc:.2f}%")

    if acc > best_acc:
        best_acc = acc
        torch.save({
            'model_state_dict': model.state_dict(),
            'label_map': label_map  # Save map for inference
        }, '/content/drive/MyDrive/bird-sounds-rnn/multi_species_model.pth')

    scheduler.step()

print(f"\nTraining complete on ALL bird species! Best val acc: {best_acc:.2f}%")
print("Model + label map saved. For inference, load label_map to get species names from predictions.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 204 unique bird species!
Classes: ['Accipiter badius', 'Acridotheres fuscus', 'Acridotheres tristis', 'Acrocephalus dumetorum', 'Acrocephalus stentoreus', 'Aegithina tiphia', 'Aethopyga vigorsii', 'Alauda gulgula', 'Alcedo atthis', 'Alcippe poioicephala', 'Amandava amandava', 'Amaurornis phoenicurus', 'Ammomanes phoenicura', 'Anas poecilorhyncha', 'Anous tenuirostris', 'Anthus campestris', 'Anthus cervinus', 'Apus affinis', 'Ardea purpurea', 'Arenaria interpres', 'Argya caudata', 'Argya malcolmi', 'Argya striata', 'Artamus fuscus', 'Athene blewitti', 'Athene brama', 'Bubo bengalensis', 'Bubulcus coromandus', 'Cacomantis passerinus', 'Cacomantis sonneratii', 'Calidris ferruginea', 'Calidris tenuirostris', 'Caprimulgus asiaticus', 'Caprimulgus atripennis', 'Caprimulgus indicus', 'Cecropis daurica', 'Centropus sinensis', 'Ceyx erithaca', 'Charadrius dubius

Epoch 1/30: 100%|██████████| 214/214 [02:08<00:00,  1.67it/s]



Epoch 1 - Loss: 2.0395 - Val Acc: 20.45%


Epoch 2/30: 100%|██████████| 214/214 [01:06<00:00,  3.22it/s]



Epoch 2 - Loss: 0.5967 - Val Acc: 21.19%


...


Epoch 28 - Loss: 0.0252 - Val Acc: 55.71%


Epoch 29/30: 100%|██████████| 214/214 [01:02<00:00,  3.41it/s]



Epoch 29 - Loss: 0.0749 - Val Acc: 56.79%


Epoch 30/30: 100%|██████████| 214/214 [01:02<00:00,  3.40it/s]



Epoch 30 - Loss: 0.0777 - Val Acc: 59.85%

Training complete on ALL bird species! Best val acc: 67.96%
Model + label map saved. For inference, load label_map to get species names from predictions.
